<a href="https://colab.research.google.com/github/Veldora00/Veldora00/blob/main/Newsandmarketsentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install newsapi-python
import requests
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from datetime import datetime, timedelta
from newsapi import NewsApiClient

# Initialize the News API client
newsapi = NewsApiClient(api_key='xxx')

# Load the FinBERT model for sentiment analysis
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')
nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# Function to get data from FRED
def get_fred_data(api_key, series_id, start_date, end_date):
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json&observation_start={start_date}&observation_end={end_date}"
    response = requests.get(url)
    data = response.json()
    return data.get('observations', [])

# Fetch Interest Rates using FRED API
def get_interest_rate(api_key, start_date, end_date):
    series_id = "DGS10"  # 10-Year Treasury Constant Maturity Rate
    data = get_fred_data(api_key, series_id, start_date, end_date)
    if not data:
        print(f"Warning: No interest rate data available for {series_id} from {start_date} to {end_date}.")
        return 0.0
    latest_rate = float(data[-1]['value'])
    return latest_rate / 100  # Convert to percentage

# Fetch Inflation Rates using FRED API
def get_inflation_rate(api_key, start_date, end_date):
    series_id = "CPIAUCSL"  # Consumer Price Index for All Urban Consumers
    data = get_fred_data(api_key, series_id, start_date, end_date)
    if len(data) < 2:
        print(f"Warning: Not enough data points returned for inflation rate. Returning default value.")
        return 0.0
    start_value = float(data[0]['value'])
    end_value = float(data[-1]['value'])
    inflation_rate = ((end_value - start_value) / start_value) * 100
    return inflation_rate

# Fetch Forex News using NewsAPI
def get_forex_news(query, from_date, to_date):
    all_articles = newsapi.get_everything(q=query,
                                          from_param=from_date,
                                          to=to_date,
                                          language='en',
                                          sort_by='relevancy')

    if all_articles['status'] == 'ok' and 'articles' in all_articles:
        return [article['title'] + ". " + article['description'] for article in all_articles['articles']]
    else:
        print(f"Error: {all_articles}")
        return []

# Analyze Sentiment using FinBERT
def analyze_sentiment(news_articles):
    sentiment_scores = []
    for article in news_articles:
        sentiment = nlp(article)
        sentiment_scores.append(sentiment[0]['score'] if sentiment[0]['label'] == 'POSITIVE' else -sentiment[0]['score'])
    return sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0

# Combine Sentiment with Macroeconomic Data
def combined_analysis(news_sentiment, interest_rate, inflation_rate, weights):
    combined_score = (
        news_sentiment * weights['sentiment'] +
        interest_rate * weights['interest_rate'] +
        inflation_rate * weights['inflation_rate']
    )
    return combined_score

# Forex Pairs (Major and Cross Pairs)
forex_pairs = [
    "EUR/USD", "GBP/USD", "USD/JPY", "AUD/USD", "USD/CHF", "USD/CAD", "NZD/USD",
    "EUR/GBP", "EUR/JPY", "EUR/CHF", "EUR/AUD", "EUR/CAD", "EUR/NZD",
    "GBP/JPY", "GBP/CHF", "GBP/AUD", "GBP/CAD", "GBP/NZD",
    "AUD/JPY", "AUD/CHF", "AUD/CAD", "AUD/NZD",
    "CHF/JPY", "CAD/JPY", "NZD/JPY",
    "CAD/CHF", "NZD/CHF", "AUD/CAD"
]

# Weights for combining sentiment and macro indicators
weights = {
    'sentiment': 0.4,
    'interest_rate': 0.35,
    'inflation_rate': 0.25
}

# Define your API keys
news_api_key = 'xxx'
fred_api_key = 'xxx'

# Define the date range for NewsAPI (limited to the past month)
news_start_date = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
news_end_date = datetime.now().strftime('%Y-%m-%d')

# Define the date range for FRED data (can be more extended)
fred_start_date = '2023-01-01'  # Example: Start of the year
fred_end_date = datetime.now().strftime('%Y-%m-%d')  # Up to the current date

# Combine Sentiment with Macroeconomic Data
def combined_analysis(news_sentiment, interest_rate, inflation_rate, weights):
    combined_score = (
        news_sentiment * weights['sentiment'] +
        interest_rate * weights['interest_rate'] +
        inflation_rate * weights['inflation_rate']
    )
    return combined_score

# Analyze each forex pair
for pair in forex_pairs:
    print(f"\nAnalyzing {pair}...")

    # Fetch news related to the forex pair
    news_articles = get_forex_news(pair, news_start_date, news_end_date)

    # Perform sentiment analysis on the news
    sentiment_score = analyze_sentiment(news_articles)
    print(f"Sentiment Score for {pair}: {sentiment_score}")

    # Get real macroeconomic indicators using FRED API
    interest_rate = get_interest_rate(fred_api_key, fred_start_date, fred_end_date)
    inflation_rate = get_inflation_rate(fred_api_key, fred_start_date, fred_end_date)

    # Combine sentiment and macroeconomic indicators
    combined_score = combined_analysis(sentiment_score, interest_rate, inflation_rate, weights)

    # Adjust the logic for determining the signal
    if combined_score > 0 and sentiment_score > 0:
        print(f"Bullish Signal for {pair}")
    elif combined_score < 0 and sentiment_score < 0:
        print(f"Bearish Signal for {pair}")
    else:
        print(f"Neutral Signal for {pair}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]


Analyzing EUR/USD...


NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [ ]:
!pip install newsapi-python
import requests
from datetime import datetime, timedelta
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# Define your API keys
alpha_vantage_api_key = 'xxx'
fred_api_key = 'xxx'

# Function to get news sentiment from Alpha Vantage
def get_alpha_vantage_news_sentiment(symbol, api_key):
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()

    if 'feed' in data:
        sentiment_scores = [float(article['overall_sentiment_score']) for article in data['feed']]
        return sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0
    else:
        print(f"Error fetching sentiment for {symbol}: {data}")
        return 0

# Function to get data from FRED
def get_fred_data(api_key, series_id, start_date, end_date):
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json&observation_start={start_date}&observation_end={end_date}"
    response = requests.get(url)
    data = response.json()
    return data.get('observations', [])

# Fetch Interest Rates using FRED API
def get_interest_rate(api_key, start_date, end_date):
    series_id = "DGS10"  # 10-Year Treasury Constant Maturity Rate
    data = get_fred_data(api_key, series_id, start_date, end_date)
    if not data:
        print(f"Warning: No interest rate data available for {series_id} from {start_date} to {end_date}.")
        return 0.0
    latest_rate = float(data[-1]['value'])
    return latest_rate / 100  # Convert to percentage

# Fetch Inflation Rates using FRED API
def get_inflation_rate(api_key, start_date, end_date):
    series_id = "CPIAUCSL"  # Consumer Price Index for All Urban Consumers
    data = get_fred_data(api_key, series_id, start_date, end_date)
    if len(data) < 2:
        print(f"Warning: Not enough data points returned for inflation rate. Returning default value.")
        return 0.0
    start_value = float(data[0]['value'])
    end_value = float(data[-1]['value'])
    inflation_rate = ((end_value - start_value) / start_value) * 100
    return inflation_rate

# Forex Pairs (Major and Cross Pairs)
forex_pairs = [
    "EUR/USD", "GBP/USD", "USD/JPY", "AUD/USD", "USD/CHF", "USD/CAD", "NZD/USD",
    "EUR/GBP", "EUR/JPY", "EUR/CHF", "EUR/AUD", "EUR/CAD", "EUR/NZD",
    "GBP/JPY", "GBP/CHF", "GBP/AUD", "GBP/CAD", "GBP/NZD",
    "AUD/JPY", "AUD/CHF", "AUD/CAD", "AUD/NZD",
    "CHF/JPY", "CAD/JPY", "NZD/JPY",
    "CAD/CHF", "NZD/CHF", "AUD/CAD"
]

# Weights for combining sentiment and macro indicators
weights = {
    'sentiment': 0.4,
    'interest_rate': 0.35,
    'inflation_rate': 0.25
}

# Define the date range for FRED data (can be more extended)
fred_start_date = '2023-01-01'  # Example: Start of the year
fred_end_date = datetime.now().strftime('%Y-%m-%d')  # Up to the current date

# Combine Sentiment with Macroeconomic Data
def combined_analysis(news_sentiment, interest_rate, inflation_rate, weights):
    combined_score = (
        news_sentiment * weights['sentiment'] +
        interest_rate * weights['interest_rate'] +
        inflation_rate * weights['inflation_rate']
    )
    return combined_score

# Analyze each forex pair
for pair in forex_pairs:
    print(f"\nAnalyzing {pair}...")

    # Fetch news sentiment from Alpha Vantage
    sentiment_score = get_alpha_vantage_news_sentiment(pair, alpha_vantage_api_key)
    print(f"Sentiment Score for {pair}: {sentiment_score}")

    # Get real macroeconomic indicators using FRED API
    interest_rate = get_interest_rate(fred_api_key, fred_start_date, fred_end_date)
    inflation_rate = get_inflation_rate(fred_api_key, fred_start_date, fred_end_date)

    # Combine sentiment and macroeconomic indicators
    combined_score = combined_analysis(sentiment_score, interest_rate, inflation_rate, weights)

    # Adjust the logic for determining the signal
    if combined_score > 0 and sentiment_score > 0:
        print(f"Bullish Signal for {pair}")
    elif combined_score < 0 and sentiment_score < 0:
        print(f"Bearish Signal for {pair}")
    else:
        print(f"Neutral Signal for {pair}")



Analyzing EUR/USD...
Error fetching sentiment for EUR/USD: {'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Sentiment Score for EUR/USD: 0
Neutral Signal for EUR/USD

Analyzing GBP/USD...
Error fetching sentiment for GBP/USD: {'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Sentiment Score for GBP/USD: 0
Neutral Signal for GBP/USD

Analyzing USD/JPY...
Error fetching sentiment for USD/JPY: {'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Se